In [6]:
import sys
sys.path.append('../')
from src.load_data import load_TEP_data_test
from src.load_data import load_TEP_data
from src.Sampler import *
from src.TEP_LOF_evaluation_func import *
import pandas as pd
from src.utils import dict_fault_ratio_fault,dict_fault_ratio_false_faults_found
import optuna

normalised=True
multiblock=True
percentage_coreset=0.5

def run_experiment(percentage_coreset):
    if multiblock:

        if not normalised:
            list_features_group=[[19, 45],
             [9, 46],
             [1, 2, 3, 4, 5, 7, 10, 13, 20, 21, 41, 42, 44],
             [8, 50],
             [17, 18, 49],
             [0, 43],
             [11, 47],
             [16, 51],
             [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
             [6, 12, 15],
             [14, 48],
             [36, 37, 38, 39, 40]]



        else:
             list_features_group = [
        [0, 43],
        [1, 2, 3, 4, 5, 7, 8, 9, 10, 13, 19, 20, 21, 22, 23, 24,
         25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
         39, 40, 41, 42, 44, 46, 50],
        [14, 16, 48, 51],
        [6, 12, 15, 45],
        [11, 47],
        [17, 18, 49]]

    else:
        list_features_group=[np.arange(0,52)]



    Training_df,Testing_df,Validation_df,cols_feature=load_TEP_data()
    scaler_list,list_corsets=generate_training_corsets(Training_df,cols_feature,list_features_group,percentage = percentage_coreset)


    # Define the function to tune
    to_tune=generate_result_validation_lof

    # Define the objective function to minimize
    def objective(trial):
        # Define the search space for the hyperparameters
        contamination = trial.suggest_uniform('contamination', 1e-6, 0.01)
        n_neighbors = trial.suggest_int('n_neighbors', 1, 200)

        # Compute the result of the function with the hyperparameters
        result = to_tune(scaler_list,list_corsets,Validation_df,cols_feature,list_features_group,n_neighbors=n_neighbors,contamination=contamination)

        # Define the target objective value
        objective_value = 1e-3

        # Return the difference between the result and the target objective value
        return abs(result - objective_value)

    # Define the Optuna study
    study = optuna.create_study(direction='minimize')

    # Optimize the hyperparameters
    study.optimize(objective, n_trials=30)

    # Print the best hyperparameters found
    print('Best hyperparameters: ', study.best_params)
    print('Best objective value: ', study.best_value)
    results_lof=test_different_fault_lof(scaler_list,list_corsets,Testing_df,Validation_df,cols_feature,list_features_group,n_neighbors=study.best_params['n_neighbors'],contamination=study.best_params['contamination'])


    dict_fault=dict_fault_ratio_fault(results_lof[1])
    df_fault = pd.DataFrame.from_dict(dict_fault, orient='index', columns=['Ratio_faults_found'])
    df_fault.to_csv('../Results/TEP/Ratio_faults_found_df_TEP_'+'LOF_'+str(percentage_coreset)+"_"+str(normalised)+"_"+str(multiblock)+'.csv')
    dict_fault=dict_fault_ratio_false_faults_found(results_lof[1])
    df_ratio_false_alarms = pd.DataFrame.from_dict(dict_fault, orient='index', columns=['Ratio_false_alarms'])
    df_ratio_false_alarms.to_csv('../Results/TEP/Ratio_false_alarms_df_TEP_'+'LOF_'+str(percentage_coreset)+"_"+str(normalised)+"_"+str(multiblock)+'.csv')


    import pickle

    # create a list

    # define the file name for the pickle file
    file_name = '../Results/TEP/Global_res_TEP_'+'LOF_'+str(percentage_coreset)+"_"+str(normalised)+"_"+str(multiblock)+'.pkl'

    # open the file in write binary mode
    with open(file_name, "wb") as f:
        # dump the list to the file using pickle
        pickle.dump(results_lof, f)



In [7]:
coreset_values = [0.1,0.4,0.7,0.9]
from joblib import Parallel, delayed


Parallel(n_jobs=-1)(delayed(run_experiment)(percentage_coreset) for percentage_coreset in coreset_values)


Subsampling...: 100%|██████████| 20250/20250 [02:11<00:00, 153.57it/s]s]
[I 2023-05-08 10:44:28,496] A new study created in memory with name: no-name-f9a790af-3e1a-4dcf-9f51-ad43eda00e8c
Subsampling...:  10%|█         | 14593/141750 [09:33<1:21:12, 26.09it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

Subsampling...:  50%|█████     | 71544/141750 [50:07<44:44, 26.15it/s]s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Subsampling...: 100%|█████████

Best hyperparameters:  {'contamination': 0.00012414292420822443, 'n_neighbors': 59}
Best objective value:  3.9999999999999996e-05


[I 2023-05-08 15:53:53,358] Trial 13 finished with value: 0.011439999999999999 and parameters: {'contamination': 0.002111980867840925, 'n_neighbors': 96}. Best is trial 12 with value: 0.00044000000000000007.
[I 2023-05-08 15:56:33,335] Trial 4 finished with value: 0.02836 and parameters: {'contamination': 0.005011978581537662, 'n_neighbors': 56}. Best is trial 3 with value: 0.00584.
[I 2023-05-08 16:05:32,863] Trial 14 finished with value: 0.01344 and parameters: {'contamination': 0.0024485574321411714, 'n_neighbors': 176}. Best is trial 12 with value: 0.00044000000000000007.
[I 2023-05-08 16:15:44,505] Trial 5 finished with value: 0.01508 and parameters: {'contamination': 0.0026952462797434776, 'n_neighbors': 54}. Best is trial 3 with value: 0.00584.
[I 2023-05-08 16:17:26,990] Trial 15 finished with value: 0.0005200000000000001 and parameters: {'contamination': 5.463028029312286e-05, 'n_neighbors': 116}. Best is trial 12 with value: 0.00044000000000000007.
[I 2023-05-08 18:20:08,403]

Best hyperparameters:  {'contamination': 0.00011893468903038593, 'n_neighbors': 72}
Best objective value:  0.00028


[I 2023-05-08 21:38:50,418] Trial 24 finished with value: 0.007200000000000001 and parameters: {'contamination': 0.0013705711117439653, 'n_neighbors': 152}. Best is trial 23 with value: 3.9999999999999996e-05.
[I 2023-05-08 21:46:04,617] Trial 25 finished with value: 0.012400000000000001 and parameters: {'contamination': 0.0022516053500060778, 'n_neighbors': 113}. Best is trial 23 with value: 3.9999999999999996e-05.
[I 2023-05-08 22:01:35,831] Trial 26 finished with value: 0.00516 and parameters: {'contamination': 0.001030341751353215, 'n_neighbors': 85}. Best is trial 23 with value: 3.9999999999999996e-05.
[I 2023-05-08 22:17:52,595] Trial 27 finished with value: 0.00304 and parameters: {'contamination': 0.0006716219005931932, 'n_neighbors': 174}. Best is trial 23 with value: 3.9999999999999996e-05.
[I 2023-05-08 22:33:28,358] Trial 28 finished with value: 0.00028 and parameters: {'contamination': 9.1964370883974e-05, 'n_neighbors': 153}. Best is trial 23 with value: 3.999999999999999

[None, None, None, None]